## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,city_id,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Vaini,TO,-21.2000,-175.2000,73.56,100,100,10.36,light rain
1,1,Kigonsera,TZ,-10.8000,35.0500,65.50,82,100,2.08,overcast clouds
2,2,Vanimo,PG,-2.6741,141.3028,83.01,73,100,4.14,overcast clouds
3,3,Noumea,NC,-22.2763,166.4572,78.78,87,24,14.36,few clouds
4,4,Comodoro Rivadavia,AR,-45.8667,-67.5000,76.86,22,0,26.46,clear sky


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,city_id,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Vaini,TO,-21.2000,-175.2000,73.56,100,100,10.36,light rain
2,2,Vanimo,PG,-2.6741,141.3028,83.01,73,100,4.14,overcast clouds
3,3,Noumea,NC,-22.2763,166.4572,78.78,87,24,14.36,few clouds
4,4,Comodoro Rivadavia,AR,-45.8667,-67.5000,76.86,22,0,26.46,clear sky
6,6,Rikitea,PF,-23.1203,-134.9692,79.12,73,4,16.69,clear sky
11,11,Samarai,PG,-10.6167,150.6667,82.63,77,92,13.58,overcast clouds
14,14,Faanui,PF,-16.4833,-151.7500,81.27,76,98,20.89,light rain
23,23,Kapaa,US,22.0752,-159.3190,73.72,82,75,13.80,broken clouds
24,24,Touros,BR,-5.1989,-35.4608,74.30,70,100,10.38,overcast clouds
27,27,Hermanus,ZA,-34.4187,19.2345,71.91,80,100,1.59,overcast clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

city_id        0
City           0
Country        5
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

city_id        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Vaini,TO,73.56,light rain,-21.2000,-175.2000,
2,Vanimo,PG,83.01,overcast clouds,-2.6741,141.3028,
3,Noumea,NC,78.78,few clouds,-22.2763,166.4572,
4,Comodoro Rivadavia,AR,76.86,clear sky,-45.8667,-67.5000,
6,Rikitea,PF,79.12,clear sky,-23.1203,-134.9692,
11,Samarai,PG,82.63,overcast clouds,-10.6167,150.6667,
14,Faanui,PF,81.27,light rain,-16.4833,-151.7500,
23,Kapaa,US,73.72,broken clouds,22.0752,-159.3190,
24,Touros,BR,74.30,overcast clouds,-5.1989,-35.4608,
27,Hermanus,ZA,71.91,overcast clouds,-34.4187,19.2345,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6d. Set up the base URL for the Google Directions API to get JSON data.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name. (Should be 13 rows)
clean_hotel_df = hotel_df.dropna()

In [14]:
# Outputting Dataframe shows gaps remain, Despite attempts to remove it.
# np.where(clean_hotel_df.applymap(lambda x: x == '')).count()
(clean_hotel_df['Hotel Name'] == None).count()

209

In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
                                 max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))